참고 문헌: 딥 러닝을 이용한 자연어 처리 입문(https://wikidocs.net/45101)

## RNN을 이용한 텍스트 생성(Text Generation using RNN)
- 다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어보자

예를 들어서 '경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장이 있다고 해보자.

모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면, 	

### X(data) / y(label)

1	경마장에	/있는  

2	경마장에 있는	/말이

3	경마장에 있는 말이	/뛰고

4	경마장에 있는 말이 뛰고	/있다

5	그의	/말이

6	그의 말이	/법이다

7	가는	/말이

8	가는 말이	/고와야

9	가는 말이 고와야	/오는

10	가는 말이 고와야 오는	/말이

11	가는 말이 고와야 오는 말이	/곱다

## 1. SimpleRNN 이용

### 1) 데이터에 대한 이해와 전처리

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

단어 집합을 생성하고 크기를 확인해보자


In [3]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1

'''
케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
'''

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
# 각 단어와 단어에 부여된 정수 인덱스 출력
print(t.word_index)


{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
# 훈련 데이터 생성
sequences = list()
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0] # 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [9]:
encoded = t.texts_to_sequences([text.split('\n')[0]])[0]
print(encoded)
print(encoded[:2])

[2, 3, 1, 4, 5]
[2, 3]


In [6]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


왜 저렇게 문장을 나눌까? 

예를 들어 **'경마장에 있는 말이 뛰고 있다'** 라는 문장 하나가 있을 때, 하나의 단어를 예측하기 위해 이전에 등장한 단어들을 모두 참고하는 것입니다.


X / y	
1	경마장에	/있는

2	경마장에 있는	/말이

3	경마장에 있는 말이	/뛰고

4	경마장에 있는 말이 뛰고 /있다

위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

In [10]:
# 전체 샘플 길이 일치
# 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이며 길이는 6

max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [11]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [12]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [14]:
# 마지막 단어를 레이블로 분리
# Numpy를 이용

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [15]:
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [16]:
# RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행 
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2) 모델 설계하기

각 단어의 임베딩 벡터는 10차원을 가지고, 32의 은닉 상태 크기를 가지는 바닐라 RNN을 사용

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
# Embedding(vocabulary 크기, 임베딩 후 벡터 크기, 입력 시퀀스 길이)
model.add(Embedding(vocab_size, 10, input_length=max_len -1 )) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # 원핫인코딩 필요
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4696 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4566 - accuracy: 0.1818
Epoch 3/200
1/1 - 0s - loss: 2.4434 - accuracy: 0.3636
Epoch 4/200
1/1 - 0s - loss: 2.4300 - accuracy: 0.4545
Epoch 5/200
1/1 - 0s - loss: 2.4163 - accuracy: 0.4545
Epoch 6/200
1/1 - 0s - loss: 2.4023 - accuracy: 0.4545
Epoch 7/200
1/1 - 0s - loss: 2.3879 - accuracy: 0.4545
Epoch 8/200
1/1 - 0s - loss: 2.3730 - accuracy: 0.4545
Epoch 9/200
1/1 - 0s - loss: 2.3576 - accuracy: 0.4545
Epoch 10/200
1/1 - 0s - loss: 2.3417 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3251 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3079 - accuracy: 0.4545
Epoch 13/200
1/1 - 0s - loss: 2.2899 - accuracy: 0.4545
Epoch 14/200
1/1 - 0s - loss: 2.2711 - accuracy: 0.4545
Epoch 15/200
1/1 - 0s - loss: 2.2516 - accuracy: 0.4545
Epoch 16/200
1/1 - 0s - loss: 2.2313 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.2101 - accuracy: 0.4545
Epoch 18/200
1/1 - 0s - loss: 2.1881 - accuracy: 0.4545
E

모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력해보자

In [20]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [21]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

경마장에 있는 말이 뛰고 있다


In [22]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [23]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


## 2. LSTM 이용

### 1) 데이터에 대한 이해와 전처리

사용할 데이터는 뉴욕 타임즈 기사의 제목 ArticlesApril2018.csv 
파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [25]:
import os

# os.environ을 이용하여 Kaggle API Username, Key 세팅하기



In [26]:
!kaggle datasets download -d aashita/nyt-comments
!unzip '*.zip'

 99% 473M/480M [00:09<00:00, 60.4MB/s]
100% 480M/480M [00:09<00:00, 53.2MB/s]
Archive:  nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [28]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [29]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [30]:
df['headline'].isnull().values.any()

False

In [31]:
# headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장

headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [32]:
# Unknown 값 -> 노이즈 데이터이므로 제거해줄 필요있음

print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [33]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [34]:
 # 전처리 수행하기
 # 구두점 제거, 단어의 소문자화
 
 def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [36]:
# 단어 집합 만들기

t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [37]:
# 훈련 데이터 구성

sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [38]:
# 어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word 만들기

index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [39]:
# 패딩 작업
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [40]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [44]:
# 레이블 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [45]:
print(y[:3]) # 레이블

[ 269  371 1115]


In [47]:
# 원핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

### 2) 모델 설계하기

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

# 각 단어의 임베딩 벡터는 10차원을 가지고, 128의 은닉 상태 크기를 가지는 LSTM
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 9s - loss: 7.6352 - accuracy: 0.0311
Epoch 2/200
244/244 - 7s - loss: 7.1152 - accuracy: 0.0292
Epoch 3/200
244/244 - 7s - loss: 6.9808 - accuracy: 0.0332
Epoch 4/200
244/244 - 6s - loss: 6.8552 - accuracy: 0.0417
Epoch 5/200
244/244 - 7s - loss: 6.7103 - accuracy: 0.0429
Epoch 6/200
244/244 - 7s - loss: 6.5464 - accuracy: 0.0452
Epoch 7/200
244/244 - 7s - loss: 6.3578 - accuracy: 0.0497
Epoch 8/200
244/244 - 7s - loss: 6.1593 - accuracy: 0.0551
Epoch 9/200
244/244 - 7s - loss: 5.9636 - accuracy: 0.0633
Epoch 10/200
244/244 - 7s - loss: 5.7772 - accuracy: 0.0641
Epoch 11/200
244/244 - 6s - loss: 5.5989 - accuracy: 0.0697
Epoch 12/200
244/244 - 6s - loss: 5.4348 - accuracy: 0.0728
Epoch 13/200
244/244 - 6s - loss: 5.2750 - accuracy: 0.0787
Epoch 14/200
244/244 - 6s - loss: 5.1258 - accuracy: 0.0852
Epoch 15/200
244/244 - 7s - loss: 4.9816 - accuracy: 0.0933
Epoch 16/200
244/244 - 7s - loss: 4.8398 - accuracy: 0.1015
Epoch 17/200
244/244 - 6s - loss: 4.7081 - accura

In [51]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [52]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i cant jump ship from facebook yet the battle of house


In [53]:
print(sentence_generation(model, t, 'how', 10))

how to serve a deranged tyrant stoically home the power of
